In [1]:
import io
from IPython.nbformat import current
def execute_notebook(nbfile):
    with io.open(nbfile) as f:
        nb = current.read(f, 'json')
    ip = get_ipython()
    for cell in nb.worksheets[0].cells:
        if cell.cell_type != 'code':
            continue
        ip.run_cell(cell.input)
execute_notebook("/cellar/users/ramarty/Projects/hla_ii/bin/imports.ipynb")
execute_notebook("/cellar/users/ramarty/Projects/hla_ii/bin/samples.ipynb")

/cellar/users/ramarty/anaconda/lib/python2.7/site-packages/IPython/nbformat/current.py:19: UserWarning: IPython.nbformat.current is deprecated.

- use IPython.nbformat for read/write/validate public API
- use IPython.nbformat.vX directly to composing notebooks of a particular version

  """)
/cellar/users/ramarty/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Populating the interactive namespace from numpy and matplotlib
Populating the interactive namespace from numpy and matplotlib


Goal: Type SRA samples with MHC-I with Polysolver and MHC-II HLA-HD <br>
and create a pipeline that will be easy to drop in new samples

Get sample names <br>
Run: python /cellar/users/ramarty/Projects/hla_ii/data_gathering/mhc_types/gather_samples.SRA.py from a qlogin session

In [45]:
samples = list(set([x.strip() for x in open('/cellar/users/ramarty/Data/hla_ii/hla_types/samples.SRA.txt').readlines()]))

In [46]:
len(samples)

839

In [6]:
samples = list(set([x.strip() for x in open('/cellar/users/ramarty/Data/hla_ii/hla_types/samples.SRA.txt').readlines() if x != '\n']))

In [7]:
samples

['SRR862173',
 'SRR309901',
 'SRR857939',
 'SRR401448',
 'SRR401445',
 'SRR860099',
 'SRR401446',
 'SRR401441',
 'SRR401440',
 'SRR310745',
 'SRR310747',
 'SRR310629',
 'SRR401527',
 'SRR860257',
 'SRR401525',
 'SRR401683',
 'SRR401682',
 'SRR401528',
 'SRR401529',
 'SRR310749',
 'SRR310627',
 'SRR401685',
 'SRR401684',
 'SRR862597',
 'SRR407060',
 'SRR863475',
 'SRR404411',
 'SRR314209',
 'SRR860950',
 'SRR860483',
 'SRR862778',
 'SRR309564',
 'SRR401510',
 'SRR314201',
 'SRR314203',
 'SRR314205',
 'SRR406612',
 'SRR309359',
 'SRR309352',
 'SRR309354',
 'SRR309355',
 'SRR862575',
 'SRR859830',
 'SRR861575',
 'SRR858768',
 'SRR860324',
 'SRR860550',
 'SRR860416',
 'SRR858443',
 'SRR401599',
 'SRR858192',
 'SRR401597',
 'SRR860795',
 'SRR401595',
 'SRR401524',
 'SRR401593',
 'SRR401591',
 'SRR862687',
 'SRR837134',
 'SRR401689',
 'SRR863496',
 'SRR837131',
 'SRR837132',
 'SRR858901',
 'SRR859084',
 'SRR405598',
 'SRR401791',
 'SRR401790',
 'SRR401793',
 'SRR401792',
 'SRR401795',
 'SRR4

Run pipeline

In [50]:
def create_cluster_script_download_and_run(samples):

    new_script_file = '/cellar/users/ramarty/Projects/hla_ii/data_gathering/mhc_types/SRA_typing.sh'

    with open(new_script_file, 'w') as out_file:
        out_file.write("#! /bin/csh\n")
        out_file.write("#$ -V\n")
        out_file.write("#$ -S /bin/csh\n")
        out_file.write("#$ -o /cellar/users/ramarty/Data/hla_ii/sge-system_files\n")
        out_file.write("#$ -e /cellar/users/ramarty/Data/hla_ii/sge-system_files\n")
        out_file.write("#$ -cwd\n")
        #out_file.write("#$ -t 1-3\n".format(len(samples)))
        out_file.write("#$ -t 1-{0}\n".format(len(samples)))
        out_file.write("#$ -l h_vmem=20G\n")
        out_file.write("#$ -tc 10\n")
        out_file.write("#$ -l long")
        out_file.write("\n")

        out_file.write("set samples=({0})\n".format(" ".join(samples)))
        out_file.write("\n")

        out_file.write("set sample=$samples[$SGE_TASK_ID]\n")
        out_file.write("\n")

        out_file.write("date\n")
        out_file.write("hostname\n")
        
        # Make output directory
        out_file.write("mkdir $sample\n")
        out_file.write("mkdir /data/nrnb03/users/ramarty/$sample\n")
        out_file.write("mkdir /tmp/$sample\n")
        out_file.write("mkdir /data/nrnb03/users/ramarty/$sample/hlaHD\n")
        out_file.write("echo New directory created.\n")
        
        # Strip section of SRA
        out_file.write("/cellar/users/ramarty/programs/sratoolkit.2.8.2-1-ubuntu64/bin/sam-dump " + 
                       "--aligned-region 6:28510120-33480577 --output-file " + 
                       "/data/nrnb03/users/ramarty/$sample/chr6.sam /data/nrnb03/dbGaP/SRA/ARIC_GoESP/$sample.sra\n")
        out_file.write("echo Sliced sam from SRA.\n")
        out_file.write("\n")
        
        # Covert and index bam
        out_file.write("source /cellar/users/ramarty/programs/polysolver/scripts/config.sh\n") 
        out_file.write("echo $SAMTOOLS_DIR\n")
        out_file.write("$SAMTOOLS_DIR/samtools view /data/nrnb03/users/ramarty/$sample/chr6.sam " + 
                       "-Sb -o /data/nrnb03/users/ramarty/$sample/chr6.bam\n")  
        out_file.write("$SAMTOOLS_DIR/samtools index /data/nrnb03/users/ramarty/$sample/chr6.bam\n")  
        out_file.write("echo Converted to bam and indexed.\n")
        out_file.write("\n")
        
        # POLYSOLVER
        #out_file.write("/cellar/users/ramarty/programs/polysolver/scripts/shell_call_hla_type " + 
        #               "/data/nrnb03/users/ramarty/$sample/chr6.bam Unknown 1 hg19 STDFQ 0 "
        #"/data/nrnb03/users/ramarty/$sample/polysolver\n")
        #out_file.write("echo Polysolver complete.\n")
        #out_file.write("\n")
        
        # Strip bam to fastq
        out_file.write("python /cellar/users/ramarty/Projects/kir/KIR_development/data_gathering/bin/convert_to_fastq2.py " + 
                       "/data/nrnb03/users/ramarty/$sample/chr6.bam /data/nrnb03/users/ramarty/$sample/merged_sorted " + 
                       "/data/nrnb03/users/ramarty/$sample/merged_1.fastq " +
                       "/data/nrnb03/users/ramarty/$sample/merged_2.fastq cellar\n")
        out_file.write("echo Fastq stripped.\n")
        out_file.write("\n")
        
        # OPTITYPE
        out_file.write("cd /cellar/users/ramarty/programs/OptiType \n")
        out_file.write("python OptiTypePipeline.py -i " +
                       "/data/nrnb03/users/ramarty/$sample/merged_1.fastq " + 
                       "/data/nrnb03/users/ramarty/$sample/merged_2.fastq " + 
                       "--dna --o /data/nrnb03/users/ramarty/$sample -v\n")
        out_file.write("echo OptiType run.\n")
        
        # Run HLA-HD
        out_file.write("hlahd.sh -t 8 -m 70 -f ~/programs/hlahd.1.0.0/freq_data/ " + 
                       "/data/nrnb03/users/ramarty/$sample/merged_1.fastq /data/nrnb03/users/ramarty/$sample/merged_2.fastq " + 
                       "~/programs/hlahd.1.0.0/HLA_gene.split.txt ~/programs/hlahd.1.0.0/dictionary/ " + 
                       "$sample /data/nrnb03/users/ramarty/$sample/hlaHD\n")
        out_file.write("echo HLA-HD completed.\n")
        out_file.write("\n")
        
        # Clean up files
        out_file.write("mv /data/nrnb03/users/ramarty/$sample/hlaHD/$sample/result/${sample}_final.result.txt " +
                       "/data/nrnb03/users/ramarty/$sample/hlaHD.txt\n")
        out_file.write("rm -r /data/nrnb03/users/ramarty/$sample/hlaHD\n")
        out_file.write("rm /data/nrnb03/users/ramarty/$sample/*sam\n")
        out_file.write("rm /data/nrnb03/users/ramarty/$sample/*bam\n")
        out_file.write("rm /data/nrnb03/users/ramarty/$sample/*bai\n")
        out_file.write("rm /data/nrnb03/users/ramarty/$sample/*fastq\n")
        out_file.write("date\n")

In [51]:
create_cluster_script_download_and_run(samples)

Extract